In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2024-07-21 02:49:19.512995: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 02:49:19.716906: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 02:49:19.716928: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 02:49:19.718012: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 02:49:19.804790: I tensorflow/core/platform/cpu_feature_g

2.14.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-21 02:49:22.005525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 02:49:22.022717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 02:49:22.022750: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Onset 모델 학습

In [1]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import Policy, set_global_policy

# 혼합 정밀도 정책 설정
policy = Policy('mixed_float16')
set_global_policy(policy)

def data_generator(x_paths, onset_paths, model):
    for x_path, onset_path in zip(x_paths, onset_paths):
        X = np.load(x_path)
        ONSET = np.load(onset_path)

        X = X / np.max(X)  # 정규화

        model.reset_states()  # 한 곡마다 모델 상태 초기화
        for x, onset in zip(X, ONSET):
            yield (x, onset)  # 한 세그먼트씩 반환

def build_model():
    input_layer = Input(batch_input_shape=(10, 100, 264), name='onset_input')
    onset_lstm = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, stateful=True, name='onset_lstm'))(input_layer)
    onset_out = TimeDistributed(Dense(88, activation='sigmoid', kernel_initializer='he_normal', name='onset_output'))(onset_lstm)
    model = Model(inputs=input_layer, outputs=onset_out)

    return model

def train(train_x, train_onset, valid_x, valid_onset):
    train_x_paths, train_onset_paths = [], []
    valid_x_paths, valid_onset_paths = [], []

    for x_path, onset_path in zip(glob.glob(train_x), glob.glob(train_onset)):
        train_x_paths.append(x_path)
        train_onset_paths.append(onset_path)

    for x_path, onset_path in zip(glob.glob(valid_x), glob.glob(valid_onset)):
        valid_x_paths.append(x_path)
        valid_onset_paths.append(onset_path)

    output_signature = (tf.TensorSpec(shape=(100, 264), dtype=tf.float32), tf.TensorSpec(shape=(100, 88), dtype=tf.int8))

    model = build_model()

    train_set = tf.data.Dataset.from_generator(lambda: data_generator(train_x_paths, train_onset_paths, model), output_signature=output_signature)
    valid_set = tf.data.Dataset.from_generator(lambda: data_generator(valid_x_paths, valid_onset_paths, model), output_signature=output_signature)
    train_set = train_set.batch(10, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    valid_set = valid_set.batch(10, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

    checkpoint = ModelCheckpoint('models/onset_detector.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    early_stop = EarlyStopping(patience=5, monitor='val_loss', verbose=1, mode='auto')

    optimizer = Adam(learning_rate=0.0005)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(train_set, validation_data=valid_set, epochs=15, shuffle=False, callbacks=[checkpoint, early_stop])
    model.save('models/onset_last.h5')

if __name__ == '__main__':
    train('data/preprocessed/trainX/*.npy', 'data/preprocessed/trainONSET/*.npy', 'data/preprocessed/validX/*.npy', 'data/preprocessed/validONSET/*.npy')


2024-07-21 12:33:23.270158: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 12:33:23.295877: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 12:33:23.295903: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 12:33:23.295914: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 12:33:23.301708: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660 SUPER, compute capability 7.5


2024-07-21 12:33:24.454535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 12:33:24.458442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 12:33:24.458467: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 12:33:24.458536: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 12:33:24.471847: I tensorflow/compile

Epoch 1/15


2024-07-21 12:33:28.232058: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-07-21 12:33:29.522798: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc4c8016830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-21 12:33:29.522830: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2024-07-21 12:33:29.532274: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-21 12:33:29.618428: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    679/Unknown - 17s 19ms/step - loss: 0.0454 - accuracy: 0.0087

KeyboardInterrupt: 

# Offset 모델 학습

In [ ]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import Policy, set_global_policy

# 혼합 정밀도 정책 설정
policy = Policy('mixed_float16')
set_global_policy(policy)

def data_generator(x_paths, onset_paths, model):
    for x_path, onset_path in zip(x_paths, onset_paths):
        X = np.load(x_path)
        ONSET = np.load(onset_path)

        X = X / np.max(X)  # 정규화

        model.reset_states()  # 한 곡마다 모델 상태 초기화
        for x, onset in zip(X, ONSET):
            yield (x, onset)  # 한 세그먼트씩 반환

def build_model():
    input_layer = Input(batch_input_shape=(10, 100, 264), name='onset_input')
    onset_lstm = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, stateful=True, name='onset_lstm'))(input_layer)
    onset_out = TimeDistributed(Dense(88, activation='sigmoid', kernel_initializer='he_normal', name='onset_output'))(onset_lstm)
    model = Model(inputs=input_layer, outputs=onset_out)

    return model

def train(train_x, train_onset, valid_x, valid_onset):
    train_x_paths, train_onset_paths = [], []
    valid_x_paths, valid_onset_paths = [], []

    for x_path, onset_path in zip(glob.glob(train_x), glob.glob(train_onset)):
        train_x_paths.append(x_path)
        train_onset_paths.append(onset_path)

    for x_path, onset_path in zip(glob.glob(valid_x), glob.glob(valid_onset)):
        valid_x_paths.append(x_path)
        valid_onset_paths.append(onset_path)

    output_signature = (tf.TensorSpec(shape=(100, 264), dtype=tf.float32), tf.TensorSpec(shape=(100, 88), dtype=tf.int8))

    model = build_model()

    train_set = tf.data.Dataset.from_generator(lambda: data_generator(train_x_paths, train_onset_paths, model), output_signature=output_signature)
    valid_set = tf.data.Dataset.from_generator(lambda: data_generator(valid_x_paths, valid_onset_paths, model), output_signature=output_signature)
    train_set = train_set.batch(10, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    valid_set = valid_set.batch(10, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

    checkpoint = ModelCheckpoint('models/offset_detector.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    early_stop = EarlyStopping(patience=5, monitor='val_loss', verbose=1, mode='auto')

    optimizer = Adam(learning_rate=0.0005)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(train_set, validation_data=valid_set, epochs=15, shuffle=False, callbacks=[checkpoint, early_stop])
    model.save('models/offset_last.h5')

if __name__ == '__main__':
    train('data/preprocessed/trainX/*.npy', 'data/preprocessed/trainOFFSET/*.npy', 'data/preprocessed/validX/*.npy', 'data/preprocessed/validOFFSET/*.npy')
